# TEI Conference 2019 #

## Etape 3 : L'Analyse ##

### Préambule ###

L'objectif présent est d'analyser les abstracts afin de les regrouper en cluster, permettant de dégager des thématiques, de comparer les différentes conférences ou bien d'étudier si les distinctions faites par les organisateurs de la conférence 2019 entre les différentes interventions est bien pertinente.
Nous pourrons également critiquer les choix de l'ordinateur.

Nous allons faire ici de la classification non supervisée grâce au package SKLearn, librairie dédiée au Machine Learning notamment.

### Les packages ### 

Il faut lancer la cellule ci-dessous une seule fois afin de télécharger la librairie sklearn. Cela doit être fait une seule fois, au premier lancement. Ensuite, il ne faudra plus jamais le lancer. Il est possible qu'il faille redémarrer le noyau après.

In [ ]:
pip install sklearn
pip install nltk

Il faut lancer la cellule ci-dessous à chaque lancement de ce notebook-ci.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import nltk
from nltk.stem import PorterStemmer
import os


Tout d'abord, il faut ' stemmer ' les abstracts, c'est à dire qu'il faut prendre chaque mot de chaque abstract et le remplacer par son lemme. Cela permettra à la machine de ne pas être faussée par l'usage divers d'un seul et même mot racine (les mots "buy", "buys", "bought" et "buying" ne deviennent plus qu'un seul et même mot). Ainsi, l'analyse n'en sera que plus précise et plus juste.

Ce texte stemmé est stocké dans le cache, dans un répertoire propre.

In [ ]:
Path = "./cache2019/cacheTXT/" #chemin permettant d'accès aux abstracts
Path_output = "./cache2019/cacheSTEM/" #chemin de sortie des abstracts stemmés
filelist = os.listdir(Path) #filelist est une liste regroupant tous les chemins vers les différents abstracts.
if not os.path.exists("./cache2019/cacheSTEM/"): #permet de créer un dossier dans le cache s'il est supprimé.
    os.makedirs("./cache2019/cacheSTEM/")

In [ ]:
stemmer = PorterStemmer() #J'importe une méthode acceptant une chaîne de caractère à une variable.
documents = [] #j'instancie une liste qui regroupera l'ensemble des chemins vers les documents
for abstract in filelist:
    with open(Path + abstract, "r", encoding="UTF-8") as y:
        documents.append(Path_output + abstract)
        texte = y.read()
        liste_mots_tokenise = [] # j'instancie une liste qui accueillera chaque lemme (le lemme étant une str)
        liste_mots_a_tokenise = texte.split() #je crée une liste sur laquelle je vais pouvoir boucler
        for elem in liste_mots_a_tokenise:
            mot_tokenise = stemmer.stem(elem) #mot_tokenise est une str, et est le lemme du mot sur lequel je boucle
            liste_mots_tokenise.append(mot_tokenise) #j'ajoute chaque mot à une liste, qu'ensuite je join pour recréer l'abstract sous la forme d'une str
        resultat = ' '.join(liste_mots_tokenise)
        with open(Path_output + abstract, "w", encoding="UTF-8") as z:
            z.write(resultat) #resultat est une str
        




Ici, on associe à chaque mot de tous les abstracts une coordonnée unique à chaque mot utilisé, pour ensuite pouvoir les placer sur un plan en 2D et ainsi relever des ressemblances.

In [ ]:
#vectorize the text i.e. convert the strings to numeric features
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(documents)

Ensuite, on réunit les termes vectorisés en cluster, c'est à dire en groupe de ressemblance.

In [ ]:
#cluster documents, ici 10 clusters
true_k = 4
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

Enfin, on montre les termes que l'ordinateur a pu rassembler.

In [ ]:
#print top terms per cluster clusters
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i)
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind])
        


En dessous, je teste des trucs, en vain. Seul le deuxième donne un truc intelligible.

In [ ]:


documents = vectorizer.transform(documents)
print(documents)

In [ ]:
z = model.fit_transform(X)
for idx, cluster in enumerate(z):
    print("Document '{}' assigned to cluster {}.".format(documents[idx],cluster))


In [ ]:
my_document_features = TfidfVectorizer().fit_transform(documents) #où chaque texte est dans la liste my_text

km = KMeans(8).fit(my_document_features)
print(km.labels_)

In [ ]:
my_document_features = vec.fit_transform('./cache2019/cacheSTEM/') #où my text files est un chemin vers un dossier où y a les textes
